In [1]:
# imports(default)
import sys
from glob import glob
import numpy as np
import tensorflow as tf
import gensim
import MeCab
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import *
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import *
keras = tf.keras

/home/shido/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
char_files = glob("../dataset/charactor/*.txt")
conv_files = glob("../dataset/conversation/*.txt")

In [3]:
char_texts = [read_file(file) for file in char_files]
conv_texts = [read_file(file) for file in conv_files]

In [4]:
parser = Parser()

In [5]:
vocab = set()
for conv in conv_texts:
    for line in conv:
        vocab = vocab.union(set(parser.parse(line)))
for conv in char_texts:
    for line in conv:
        vocab = vocab.union(set(parser.parse(line)))

In [6]:
word2id = {}
id2word = {}

for e, word in enumerate(["<PAD>", "<BOS>", "<EOS>", "<UNK>"] + sorted(list(vocab))):
    word2id[word] = e
    id2word[e] = word

In [7]:
tokenizer = Tokenizer(word2id, id2word)
char_seqs = [[tokenizer.encode(parser.parse(line)) for line in text] for text in char_texts]
conv_seqs = [[tokenizer.encode(parser.parse(line)) for line in text] for text in conv_texts]

In [8]:
conv_x = []
conv_y = []

In [9]:
for text in conv_seqs:
    conv_x += text[0::2]
    conv_y += text[1::2]

In [10]:
EMBEDDING_SIZE = 200
NUM_UNITS = 400
SEQ_LEN = 150
BEAM_WIDTH = 3
BATCH_SIZE = 256
VOCAB = len(word2id)

In [ ]:
from tensorflow.python.keras.callbacks import *

x, y = get_batch(conv_x, conv_y, SEQ_LEN)
clbk = [ModelCheckpoint("../models/weights.{epoch:03d}.hdf5", period=10)]
model.training_model.fit(x, y[:,:,np.newaxis], 64, epochs=100, callbacks=clbk, shuffle=True, initial_epoch=10)

In [13]:
model.training_model.load_weights("../models/weights.100.hdf5")

In [14]:
i = 111
source = conv_x[i:i+2]
inputs = pad_sequences(source, 150, padding="post", truncating="post")
states = model.predict_sequence(inputs, 50, mode="random", alpha=1)
print(tokenizer.decode_batch(source))
print(tokenizer.decode_batch(states))

['どんなジャンルの音楽が好きですか？', 'どんなジャンルが好きなんですか？']
['海に行かないんですか？', 'すごく簡単です。']


In [20]:
query = "明日は何をしますか"
print(parser.parse(query))
query = [tokenizer.encode(parser.parse(query))]
print(tokenizer.decode(query))
query = pad_sequences(query, 150, padding="post", truncating="post")
result = model.predict_sequence(query, 50, mode="random", alpha=.8)
print(tokenizer.decode(result[0]))

['明日', 'は', '何', 'を', 'し', 'ます', 'か']

ぼくはペットへ大好きです
